# Montu Python 
## Astronomical ephemerides for the Ancient World
## Adapting HYG Catalogu to `MontuPython`


In [1]:
from montu import *

In [59]:
greek_letters = {
    'α':'Alp','β':'Bet','χ':'Chi','δ':'Del','ε':'Eps','η':'Eta','γ':'Gam',
    'ι':'Iot','κ':'Kap','λ':'Lam','μ':'Mu','ν':'Nu','ω':'Ome','ο':'Omi','φ':'Phi','π':'Pi',
    'ψ':'Psi','ρ':'Rho','σ':'Sig','τ':'Tau','θ':'The','υ':'Ups','ξ':'Xi','ζ':'Zet',
}
letters_greek = dict()
for let,abrev in greek_letters.items():
    letters_greek[abrev] = let

### HYG Catalogue

This will the the official source of `MontuPython`: 

- WebPage: http://www.astronexus.com/hyg, 
- GitHub: https://github.com/astronexus/HYG-Database/tree/master/hyg 

## Get the latest version of the Catalogue

In [2]:
Montu._wget('https://raw.githubusercontent.com/astronexus/HYG-Database/master/hyg/v3/hyg_v37.csv',filename='catalogues/hyg_v37.csv')

32.4MB [00:00, 56.0MB/s]                            


In [3]:
hyg_cat = pd.read_csv('catalogues/hyg_v37.csv')
hyg_cat.sort_values(by='mag',inplace=True)

In [6]:
print("Catalogue completeness:")
nstars = len(hyg_cat)
columns = hyg_cat.columns
for col in columns:
    ncomplete = (~(hyg_cat[col].isnull())).sum()
    print(f"\t{col}: {ncomplete}/{nstars} ({ncomplete/nstars*100:.1f}%)")

Catalogue completeness:
	id: 119626/119626 (100.0%)
	hip: 117951/119626 (98.6%)
	hd: 98885/119626 (82.7%)
	hr: 9041/119626 (7.6%)
	gl: 3801/119626 (3.2%)
	bf: 3099/119626 (2.6%)
	proper: 367/119626 (0.3%)
	ra: 119626/119626 (100.0%)
	dec: 119626/119626 (100.0%)
	dist: 119626/119626 (100.0%)
	pmra: 119626/119626 (100.0%)
	pmdec: 119626/119626 (100.0%)
	rv: 119626/119626 (100.0%)
	mag: 119626/119626 (100.0%)
	absmag: 119626/119626 (100.0%)
	spect: 116578/119626 (97.5%)
	ci: 117735/119626 (98.4%)
	x: 119626/119626 (100.0%)
	y: 119626/119626 (100.0%)
	z: 119626/119626 (100.0%)
	vx: 119626/119626 (100.0%)
	vy: 119626/119626 (100.0%)
	vz: 119626/119626 (100.0%)
	rarad: 119626/119626 (100.0%)
	decrad: 119626/119626 (100.0%)
	pmrarad: 119626/119626 (100.0%)
	pmdecrad: 119626/119626 (100.0%)
	bayer: 1537/119626 (1.3%)
	flam: 2737/119626 (2.3%)
	con: 119625/119626 (100.0%)
	comp: 119626/119626 (100.0%)
	comp_primary: 119626/119626 (100.0%)
	base: 1086/119626 (0.9%)
	lum: 119626/119626 (100.0%)
	

In [7]:
Montu.print_df(hyg_cat.head(10))

,id,hip,hd,hr,gl,bf,proper,ra,dec,dist,pmra,pmdec,rv,mag,absmag,spect,ci,x,y,z,vx,vy,vz,rarad,decrad,pmrarad,pmdecrad,bayer,flam,con,comp,comp_primary,base,lum,var,var_min,var_max
0,0,NaN,NaN,NaN,NaN,NaN,Sol,0.00000000000000000,0.00000000000000000,0.00000000000000000,0.00000000000000000,0.00000000000000000,0.00000000000000000,-26.69999999999999929,4.84999999999999964,G2V,0.65600000000000003,0.00000500000000000,0.00000000000000000,0.00000000000000000,0.00000000,0.00000000,0.00000000,0.00000000000000000,0.00000000000000000,0.0000000000000000,0.000000000000000,NaN,NaN,NaN,1,0,NaN,1.00000000000000000,NaN,NaN,NaN
32263,32263,32349.0,48915.0,2491.0,Gl 244A,9Alp CMa,Sirius,6.75248100000000040,-16.71611599999999953,2.63710000000000022,-546.00999999999999091,-1223.07999999999992724,-9.40000000000000036,-1.43999999999999995,1.45399999999999996,A0m...,0.00900000000000000,-0.49432300000000001,2.47673100000000002,-0.75848499999999996,0.00000953,-0.00001207,-0.00001221,1.76779536960219952,-0.29175125851768502,-0.0000026471311772,-0.000005929659164,Alp,9.0,CMa,1,32263,Gl 244,22.82443312173503713,NaN,-1.33299999999999996,-1.52299999999999991
30365,30365,30438.0,45348.0,2326.0,NaN,Alp Car,Canopus,6.39919499999999974,-52.69565999999999661,94.78669999999999618,19.98999999999999844,23.67000000000000171,21.00000000000000000,-0.62000000000000000,-5.50399999999999956,F0Ib,0.16400000000000001,-5.99267899999999987,57.13203399999999732,-75.39610500000000570,-0.00001140,0.00002059,-0.00001049,1.67530535199972919,-0.91971277489026759,0.0000000969142547,0.000000114755398,Alp,NaN,Car,1,30365,NaN,13854.79167566721662297,NaN,-0.54700000000000004,-0.66700000000000004
69450,69451,69673.0,124897.0,5340.0,Gl 541,16Alp Boo,Arcturus,14.26102999999999987,19.18241000000000085,11.25750000000000028,-1093.45000000000004547,-1999.40000000000009095,-5.00000000000000000,-0.05000000000000000,-0.30700000000000000,K2IIIp,1.23900000000000010,-8.82344800000000085,-5.93253100000000000,3.69893800000000006,-0.00005905,0.00003220,-0.00010473,3.73352902854208724,0.33479621805862120,-0.0000053011951900,-0.000009693364729,Alp,16.0,Boo,1,69451,NaN,115.55799554021291442,NaN,NaN,NaN
71455,71456,71683.0,128620.0,5459.0,Gl 559A,Alp1Cen,Rigil Kentaurus,14.66076499999999960,-60.83397599999999983,1.32479999999999998,-3678.19000000000005457,481.83999999999997499,-26.19999999999999929,-0.01000000000000000,4.37899999999999956,G2V,0.70999999999999996,-0.49520300000000000,-0.41408400000000001,-1.15662500000000001,-0.00000722,0.00002476,0.00002491,3.83817921177722798,-1.06175317577041750,-0.0000178323683168,0.000002336026238,Alp-1,NaN,Cen,1,71456,Gl 559,1.54312106565779206,NaN,0.11300000000000000,-0.08699999999999999
90978,90979,91262.0,172167.0,7001.0,Gl 721,3Alp Lyr,Vega,18.61563999999999908,38.78369200000000205,7.67870000000000008,201.02000000000001023,287.45999999999997954,-12.09999999999999964,0.03000000000000000,0.60399999999999998,A0Vvar,-0.00100000000000000,0.96056500000000000,-5.90800899999999984,4.80973100000000020,0.00000476,0.00001734,0.00000059,4.87356309550903077,0.67690311639730250,0.0000009745724607,0.000001393645406,Alp,3.0,Lyr,1,90979,NaN,49.93441887213497665,NaN,NaN,NaN
24549,24549,24608.0,34029.0,1708.0,Gl 194A,13Alp Aur,Capella,5.27815000000000012,45.99799099999999896,13.12340000000000018,75.51999999999999602,-427.12999999999999545,22.19999999999999929,0.08000000000000000,-0.51000000000000001,M1: comp,0.79500000000000004,1.71263300000000007,8.95443900000000070,9.44000699999999959,0.00000192,0.00003559,-0.00000254,1.38181641264880106,0.80281639263411120,0.0000003661312916,-0.000002070784673,Alp,13.0,Aur,1,24549,Gl 194,139.31568029453021040,NaN,NaN,NaN
24378,24378,24436.0,34085.0,1713.0,NaN,19Bet Ori,Rigel,5.24229799999999990,-8.20163999999999938,264.55029999999999291,1.87000000000000011,-0.56000000000000005,21.00000000000000000,0.17999999999999999,-6.93299999999999983,B8Ia,-0.03000000000000000,51.60110600000000147,256.70990499999999201,-37.74005100000000112,0.00000182,0.0000212

Make the adjustments:

In [56]:
def change_UTF(str):
    def sub(m):
        subs = letters_greek[m.group(1)]+m.group(2)
        return subs
    utf = re.sub('(\w+)[-]*([\s]*.*)',sub,str)
    return utf

In [96]:
# Read
hyg_cat = pd.read_csv('catalogues/hyg_v37.csv')
hyg_cat.sort_values(by=['mag','ra'],inplace=True)

# Drop columns
hyg_cat.reset_index(inplace=True)
hyg_cat['MN'] = hyg_cat.apply(lambda x:int(x.name)+1,axis=1)
hyg_cat['MN_Name'] = hyg_cat['MN'].apply(lambda x:f'MN {x}')

# Rename columns
hyg_cat.rename(columns={
    'hip':'HIP','hd':'HD','hr':'HR','gl':'Gl','bf':'BF','proper':'ProperName',
    'ra':'RAJ2000','dec':'DecJ2000','pmra':'pmRA','pmdec':'pmDec',
    'rv':'RadVel','mag':'Vmag','absmag':'Mag','spect':'SpType','ci':'B-V',
    'x':'XJ2000','y':'YJ2000','z':'ZJ2000',
    'vx':'VZJ2000','vy':'VYJ2000','vz':'VXJ2000',
    'bayer':'Bayer','flam':'Flamsteed','con':'Constellation',
    'lum':'Luminosity','index':'HYG','dist':'Distance',
    'var_min':'Vmag_min','var_max':'Vmag_max',
    'comp_primary':'Primary','base':'MultipleID',
},inplace=True)

# Conver catalogue to integers and fill with 0
for cat_field in 'HIP','HD','HR':
    hyg_cat[cat_field] = hyg_cat[cat_field].fillna(0).astype(int)

# Adjust numeric values
hyg_cat['RAJ2000'] = hyg_cat['RAJ2000'].apply(lambda x:15*x)

# Multiple and variable
hyg_cat['IsMultiple'] = 0
cond = hyg_cat['MultipleID'].isna()
hyg_cat.loc[~cond,'IsMultiple'] = 1
hyg_cat['IsVariable'] = 0
cond = (~hyg_cat['Vmag_min'].isna())|(~hyg_cat['var'].isna())
hyg_cat.loc[cond,'IsVariable'] = 1

# Adjdust Bayer
cond = (~hyg_cat.Bayer.isna())
hyg_cat.loc[cond,'Bayer'] = hyg_cat.loc[cond].apply(lambda x:change_UTF(x['Bayer'])+' '+x['Constellation'],axis=1)

# Adjdust Flamsteed
cond = (~hyg_cat.Flamsteed.isna())
hyg_cat.loc[cond,'Flamsteed'] = hyg_cat.loc[cond].apply(lambda x:str(int(x['Flamsteed']))+' '+x['Constellation'],axis=1)

# Name
hyg_cat['Name'] = hyg_cat['ProperName'].fillna(hyg_cat['Bayer'])

# Other Designations
hyg_cat['OtherDesignations'] = hyg_cat['Name']

# Columns
# Output columns
columns = ['MN','HD', 'HR', 'HIP', 'Gl', 'Name', 'OtherDesignations', 'ProperName', 'Bayer', 'Flamsteed', 'Constellation',
           'RAJ2000', 'DecJ2000', 'pmRA', 'pmDec', 'RadVel', 'Distance',
           'Vmag','Vmag_min','Vmag_max','B-V','SpType','Luminosity',
           'XJ2000','YJ2000','ZJ2000','VXJ2000','VYJ2000','VZJ2000',
           'Primary','MultipleID',
           'IsMultiple','IsVariable']
montu_cat = hyg_cat[columns]
Montu.print_df(hyg_cat.head(10))
Montu.print_df(montu_cat.head(10))

,HYG,id,HIP,HD,HR,Gl,BF,ProperName,RAJ2000,DecJ2000,Distance,pmRA,pmDec,RadVel,Vmag,Mag,SpType,B-V,XJ2000,YJ2000,ZJ2000,VZJ2000,VYJ2000,VXJ2000,rarad,decrad,pmrarad,pmdecrad,Bayer,Flamsteed,Constellation,comp,Primary,MultipleID,Luminosity,var,Vmag_min,Vmag_max,MN,MN_Name,IsMultiple,IsVariable,Name,OtherDesignations
0,0,0,0,0,0,NaN,NaN,Sol,0.00000000000000000,0.00000000000000000,0.00000000000000000,0.00000000000000000,0.00000000000000000,0.00000000000000000,-26.69999999999999929,4.84999999999999964,G2V,0.65600000000000003,0.00000500000000000,0.00000000000000000,0.00000000000000000,0.00000000,0.00000000,0.00000000,0.00000000000000000,0.00000000000000000,0.0000000000000000,0.000000000000000,NaN,NaN,NaN,1,0,NaN,1.00000000000000000,NaN,NaN,NaN,1,MN 1,0,0,Sol,Sol
1,32263,32263,32349,48915,2491,Gl 244A,9Alp CMa,Sirius,101.28721500000000333,-16.71611599999999953,2.63710000000000022,-546.00999999999999091,-1223.07999999999992724,-9.40000000000000036,-1.43999999999999995,1.45399999999999996,A0m...,0.00900000000000000,-0.49432300000000001,2.47673100000000002,-0.75848499999999996,0.00000953,-0.00001207,-0.00001221,1.76779536960219952,-0.29175125851768502,-0.0000026471311772,-0.000005929659164,α CMa,9 CMa,CMa,1,32263,Gl 244,22.82443312173503713,NaN,-1.33299999999999996,-1.52299999999999991,2,MN 2,1,1,Sirius,Sirius
2,30365,30365,30438,45348,2326,NaN,Alp Car,Canopus,95.98792499999998995,-52.69565999999999661,94.78669999999999618,19.98999999999999844,23.67000000000000171,21.00000000000000000,-0.62000000000000000,-5.50399999999999956,F0Ib,0.16400000000000001,-5.99267899999999987,57.13203399999999732,-75.39610500000000570,-0.00001140,0.00002059,-0.00001049,1.67530535199972919,-0.91971277489026759,0.0000000969142547,0.000000114755398,α Car,NaN,Car,1,30365,NaN,13854.79167566721662297,NaN,-0.54700000000000004,-0.66700000000000004,3,MN 3,0,1,Canopus,Canopus
3,69450,69451,69673,124897,5340,Gl 541,16Alp Boo,Arcturus,213.91544999999999277,19.18241000000000085,11.25750000000000028,-1093.45000000000004547,-1999.40000000000009095,-5.00000000000000000,-0.05000000000000000,-0.30700000000000000,K2IIIp,1.23900000000000010,-8.82344800000000085,-5.93253100000000000,3.69893800000000006,-0.00005905,0.00003220,-0.00010473,3.73352902854208724,0.33479621805862120,-0.0000053011951900,-0.000009693364729,α Boo,16 Boo,Boo,1,69451,NaN,115.55799554021291442,NaN,NaN,NaN,4,MN 4,0,0,Arcturus,Arcturus
4,71455,71456,71683,128620,5459,Gl 559A,Alp1Cen,Rigil Kentaurus,219.91147499999999582,-60.83397599999999983,1.32479999999999998,-3678.19000000000005457,481.83999999999997499,-26.19999999999999929,-0.01000000000000000,4.37899999999999956,G2V,0.70999999999999996,-0.49520300000000000,-0.41408400000000001,-1.15662500000000001,-0.00000722,0.00002476,0.00002491,3.83817921177722798,-1.06175317577041750,-0.0000178323683168,0.000002336026238,α1 Cen,NaN,Cen,1,71456,Gl 559,1.54312106565779206,NaN,0.11300000000000000,-0.08699999999999999,5,MN 5,1,1,Rigil Kentaurus,Rigil Kentaurus
5,90978,90979,91262,172167,7001,Gl 721,3Alp Lyr,Vega,279.23460000000000036,38.78369200000000205,7.67870000000000008,201.02000000000001023,287.45999999999997954,-12.09999999999999964,0.03000000000000000,0.60399999999999998,A0Vvar,-0.00100000000000000,0.96056500000000000,-5.90800899999999984,4.80973100000000020,0.00000476,0.00001734,0.00000059,4.87356309550903077,0.67690311639730250,0.0000009745724607,0.000001393645406,α Lyr,3 Lyr,Lyr,1,90979,NaN,49.93441887213497665,NaN,NaN,NaN,6,MN 6,0,0,Vega,Vega
6,24549,24549,24608,34029,1708,Gl 194A,13Alp Aur,Capella,79.17225000000000534,45.99799099999999896,13.12340000000000018,75.51999999999999602,-427.12999999999999545,22.19999999999999929,0.08000000000000000,-0.51000000000000001,M1: comp,0.79500000000000004,1.71263300000000007,8.95443900000000070,9.44000699999999959,0.00000192,0.00003559,-0.00000254,1.38181641264880106,0.80281639263411120,0.0000003661312916,-0.000002070784673,α Aur,13 Aur,Aur,1,24549,Gl 194,139.31568029453021040,NaN,NaN,NaN,7,MN 7,1,0,Capella,Capella
7,2437

,MN,HD,HR,HIP,Gl,Name,OtherDesignations,ProperName,Bayer,Flamsteed,Constellation,RAJ2000,DecJ2000,pmRA,pmDec,RadVel,Distance,Vmag,Vmag_min,Vmag_max,B-V,SpType,Luminosity,XJ2000,YJ2000,ZJ2000,VXJ2000,VYJ2000,VZJ2000,Primary,MultipleID,IsMultiple,IsVariable
0,1,0,0,0,NaN,Sol,Sol,Sol,NaN,NaN,NaN,0.00000000000000000,0.00000000000000000,0.00000000000000000,0.00000000000000000,0.00000000000000000,0.00000000000000000,-26.69999999999999929,NaN,NaN,0.65600000000000003,G2V,1.00000000000000000,0.00000500000000000,0.00000000000000000,0.00000000000000000,0.00000000,0.00000000,0.00000000,0,NaN,0,0
1,2,48915,2491,32349,Gl 244A,Sirius,Sirius,Sirius,α CMa,9 CMa,CMa,101.28721500000000333,-16.71611599999999953,-546.00999999999999091,-1223.07999999999992724,-9.40000000000000036,2.63710000000000022,-1.43999999999999995,-1.33299999999999996,-1.52299999999999991,0.00900000000000000,A0m...,22.82443312173503713,-0.49432300000000001,2.47673100000000002,-0.75848499999999996,-0.00001221,-0.00001207,0.00000953,32263,Gl 244,1,1
2,3,45348,2326,30438,NaN,Canopus,Canopus,Canopus,α Car,NaN,Car,95.98792499999998995,-52.69565999999999661,19.98999999999999844,23.67000000000000171,21.00000000000000000,94.78669999999999618,-0.62000000000000000,-0.54700000000000004,-0.66700000000000004,0.16400000000000001,F0Ib,13854.79167566721662297,-5.99267899999999987,57.13203399999999732,-75.39610500000000570,-0.00001049,0.00002059,-0.00001140,30365,NaN,0,1
3,4,124897,5340,69673,Gl 541,Arcturus,Arcturus,Arcturus,α Boo,16 Boo,Boo,213.91544999999999277,19.18241000000000085,-1093.45000000000004547,-1999.40000000000009095,-5.00000000000000000,11.25750000000000028,-0.05000000000000000,NaN,NaN,1.23900000000000010,K2IIIp,115.55799554021291442,-8.82344800000000085,-5.93253100000000000,3.69893800000000006,-0.00010473,0.00003220,-0.00005905,69451,NaN,0,0
4,5,128620,5459,71683,Gl 559A,Rigil Kentaurus,Rigil Kentaurus,Rigil Kentaurus,α1 Cen,NaN,Cen,219.91147499999999582,-60.83397599999999983,-3678.19000000000005457,481.83999999999997499,-26.19999999999999929,1.32479999999999998,-0.01000000000000000,0.11300000000000000,-0.08699999999999999,0.70999999999999996,G2V,1.54312106565779206,-0.49520300000000000,-0.41408400000000001,-1.15662500000000001,0.00002491,0.00002476,-0.00000722,71456,Gl 559,1,1
5,6,172167,7001,91262,Gl 721,Vega,Vega,Vega,α Lyr,3 Lyr,Lyr,279.23460000000000036,38.78369200000000205,201.02000000000001023,287.45999999999997954,-12.09999999999999964,7.67870000000000008,0.03000000000000000,NaN,NaN,-0.00100000000000000,A0Vvar,49.93441887213497665,0.96056500000000000,-5.90800899999999984,4.80973100000000020,0.00000059,0.00001734,0.00000476,90979,NaN,0,0
6,7,34029,1708,24608,Gl 194A,Capella,Capella,Capella,α Aur,13 Aur,Aur,79.17225000000000534,45.99799099999999896,75.51999999999999602,-427.12999999999999545,22.19999999999999929,13.12340000000000018,0.08000000000000000,NaN,NaN,0.79500000000000004,M1: comp,139.31568029453021040,1.71263300000000007,8.95443900000000070,9.44000699999999959,-0.00000254,0.00003559,0.00000192,24549,Gl 194,1,0
7,8,34085,1713,24436,NaN,Rigel,Rigel,Rigel,β Ori,19 Ori,Ori,78.63446999999999321,-8.20163999999999938,1.87000000000000011,-0.56000000000000005,21.00000000000000000,264.55029999999999291,0.17999999999999999,NaN,NaN,-0.03000000000000000,B8Ia,51665.42425669496878982,51.60110600000000147,256.70990499999999201,-37.74005100000000112,-0.00000377,0.00002121,0.00000182,24378,NaN,0,0
8,9,61421,2943,37279,Gl 280A,Procyon,Procyon,Procyon,α CMi,10 CMi,CMi,114.82549500000000364,5.22499300000000044,-716.57000000000005002,-1034.57999999999992724,-4.00000000000000000,3.51420000000000021,0.40000000000000002,NaN,NaN,0.43200000000000000,F5IV-V,7.44046366191919351,-1.46931199999999995,3.17616999999999994,0.32002399999999998,-0.00001792,0.00000289,0.00001212,37173,Gl 280,1,0
9,10,10144,472,7588,NaN,Achernar,Achernar,Achernar,α Eri,NaN,Eri,24.42833999999999861,-57.23675699999999722,88.01999999999999602,-40.07999999999999829,16.00000000000000000,42.75330000000000297,0.4500000000

In [76]:
#np.unique(hyg_cat.Bayer.dropna())

In [71]:
cond = (~hyg_cat.Bayer.isna())
hyg_cat.loc[cond,'Bayer'] = hyg_cat.loc[cond,'Bayer'].apply(change_UTF)

In [78]:
Montu.print_df(hyg_cat.head(20))

,id,HIP,HD,HR,Gl,BF,ProperName,RAJ2000,DecJ2000,dist,pmRA,pmDec,RadVel,Vmag,Mag,SpType,B-V,XJ2000,XJ2000,XJ2000,VZJ2000,VYJ2000,VXJ2000,rarad,decrad,pmrarad,pmdecrad,Bayer,Flamsteed,Constellation,comp,comp_primary,base,Luminosity,var,var_min,var_max,IsMultiple,IsVariable
0,0,0,0,0,NaN,NaN,Sol,0.00000000000000000,0.00000000000000000,0.00000000000000000,0.00000000000000000,0.00000000000000000,0.00000000000000000,-26.69999999999999929,4.84999999999999964,G2V,0.65600000000000003,0.00000500000000000,0.00000000000000000,0.00000000000000000,0.00000000,0.00000000,0.00000000,0.00000000000000000,0.00000000000000000,0.0000000000000000,0.000000000000000,NaN,NaN,NaN,1,0,NaN,1.00000000000000000,NaN,NaN,NaN,0,0
32263,32263,32349,48915,2491,Gl 244A,9Alp CMa,Sirius,101.28721500000000333,-16.71611599999999953,2.63710000000000022,-546.00999999999999091,-1223.07999999999992724,-9.40000000000000036,-1.43999999999999995,1.45399999999999996,A0m...,0.00900000000000000,-0.49432300000000001,2.47673100000000002,-0.75848499999999996,0.00000953,-0.00001207,-0.00001221,1.76779536960219952,-0.29175125851768502,-0.0000026471311772,-0.000005929659164,α CMa,9.0,CMa,1,32263,Gl 244,22.82443312173503713,NaN,-1.33299999999999996,-1.52299999999999991,1,0
30365,30365,30438,45348,2326,NaN,Alp Car,Canopus,95.98792499999998995,-52.69565999999999661,94.78669999999999618,19.98999999999999844,23.67000000000000171,21.00000000000000000,-0.62000000000000000,-5.50399999999999956,F0Ib,0.16400000000000001,-5.99267899999999987,57.13203399999999732,-75.39610500000000570,-0.00001140,0.00002059,-0.00001049,1.67530535199972919,-0.91971277489026759,0.0000000969142547,0.000000114755398,α Car,NaN,Car,1,30365,NaN,13854.79167566721662297,NaN,-0.54700000000000004,-0.66700000000000004,0,0
69450,69451,69673,124897,5340,Gl 541,16Alp Boo,Arcturus,213.91544999999999277,19.18241000000000085,11.25750000000000028,-1093.45000000000004547,-1999.40000000000009095,-5.00000000000000000,-0.05000000000000000,-0.30700000000000000,K2IIIp,1.23900000000000010,-8.82344800000000085,-5.93253100000000000,3.69893800000000006,-0.00005905,0.00003220,-0.00010473,3.73352902854208724,0.33479621805862120,-0.0000053011951900,-0.000009693364729,α Boo,16.0,Boo,1,69451,NaN,115.55799554021291442,NaN,NaN,NaN,0,0
71455,71456,71683,128620,5459,Gl 559A,Alp1Cen,Rigil Kentaurus,219.91147499999999582,-60.83397599999999983,1.32479999999999998,-3678.19000000000005457,481.83999999999997499,-26.19999999999999929,-0.01000000000000000,4.37899999999999956,G2V,0.70999999999999996,-0.49520300000000000,-0.41408400000000001,-1.15662500000000001,-0.00000722,0.00002476,0.00002491,3.83817921177722798,-1.06175317577041750,-0.0000178323683168,0.000002336026238,α1 Cen,NaN,Cen,1,71456,Gl 559,1.54312106565779206,NaN,0.11300000000000000,-0.08699999999999999,1,0
90978,90979,91262,172167,7001,Gl 721,3Alp Lyr,Vega,279.23460000000000036,38.78369200000000205,7.67870000000000008,201.02000000000001023,287.45999999999997954,-12.09999999999999964,0.03000000000000000,0.60399999999999998,A0Vvar,-0.00100000000000000,0.96056500000000000,-5.90800899999999984,4.80973100000000020,0.00000476,0.00001734,0.00000059,4.87356309550903077,0.67690311639730250,0.0000009745724607,0.000001393645406,α Lyr,3.0,Lyr,1,90979,NaN,49.93441887213497665,NaN,NaN,NaN,0,0
24549,24549,24608,34029,1708,Gl 194A,13Alp Aur,Capella,79.17225000000000534,45.99799099999999896,13.12340000000000018,75.51999999999999602,-427.12999999999999545,22.19999999999999929,0.08000000000000000,-0.51000000000000001,M1: comp,0.79500000000000004,1.71263300000000007,8.95443900000000070,9.44000699999999959,0.00000192,0.00003559,-0.00000254,1.38181641264880106,0.80281639263411120,0.0000003661312916,-0.000002070784673,α Aur,13.0,Aur,1,24549,Gl 194,139.31568029453021040,NaN,NaN,NaN,1,0
24378,24378,24436,34085,1713,NaN,19Bet Ori,Rigel,78.63446999999999321,-8.20163999999999938,264.55029999999999291,1.87000000000000011,-0.56000000000000005,21.00000000000000000,0.17999999999999999,-6.93299999999999983,B8Ia,-0.030000000000